# Heston Almost Exact Simulation

This is a qablet adaptation of [Heston Almost Exact Simulation](https://github.com/nburgessx/Papers/tree/main/HestonSimulation) by Nicholas Burgess.

In [ ]:
import numpy as np
from datetime import datetime
from math import sqrt
import pyarrow as pa

from qablet.base.mc import MCModel, MCStateBase  # The two base classes we need for our custom model
from numpy.random import Generator, SFC64
from qablet.base.utils import Forwards, discounter_from_dataset
from qablet_contracts.eq.vanilla import Option
from qablet_contracts.timetable import TS_EVENT_SCHEMA, py_to_ts
from qablet.base.flags import Stats

## Create the Model State Class

In [ ]:
def CIR_Sample(NoOfPaths, kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 * kappa * vbar / gamma / gamma
    c = 1.0 / (4.0 * kappa) * gamma * gamma * (1.0 - np.exp(-kappa * (t - s)))
    kappaBar = (
        4.0
        * kappa
        * v_s
        * np.exp(-kappa * (t - s))
        / (gamma * gamma * (1.0 - np.exp(-kappa * (t - s))))
    )
    sample = c * np.random.noncentral_chisquare(delta, kappaBar, NoOfPaths)
    return sample


class HestonAESMCState(MCStateBase):
    def __init__(self, timetable, dataset):
        super().__init__(timetable, dataset)

        # fetch the model parameters from the dataset
        self.n = dataset["MC"]["PATHS"]
        self.asset = dataset["HESTON"]["ASSET"]
        self.asset_fwd = Forwards(dataset["ASSETS"][self.asset])
        self.spot = self.asset_fwd.forward(0)

        self.gamma = dataset["HESTON"]["VOL_OF_VAR"]
        self.kappa = dataset["HESTON"]["MEANREV"]
        self.vbar = dataset["HESTON"]["LONG_VAR"]
        self.rho = dataset["HESTON"]["CORRELATION"]
        self.v = dataset["HESTON"]["INITIAL_VAR"]

        # Initialize the arrays
        self.rng = Generator(SFC64(dataset["MC"]["SEED"]))
        self.x_vec = np.zeros(self.n)  # process x (log stock)

        self.cur_time = 0

    def advance(self, new_time):
        """Update x_vec in place when we move simulation by time dt."""

        dt = new_time - self.cur_time
        if dt < 1e-10:
            return

        r = self.asset_fwd.rate(new_time, self.cur_time)

        # Generate the Brownian Increments
        dw_vec = self.rng.standard_normal(self.n) * sqrt(dt)  # * self.vol

        # Exact samples for the variance process
        new_v = CIR_Sample(self.n, self.kappa, self.gamma, self.vbar, 0, dt, self.v)

        # AES Constant Terms
        k0 = (r - self.rho / self.gamma * self.kappa * self.vbar) * dt
        k1 = (self.rho * self.kappa / self.gamma - 0.5) * dt - self.rho / self.gamma
        k2 = self.rho / self.gamma

        # Almost Exact Simulation for Log-Normal Asset Process
        self.x_vec += (
            k0
            + k1 * self.v
            + k2 * new_v
            + np.sqrt((1.0 - self.rho**2) * self.v) * dw_vec
        )

        self.v = new_v
        self.cur_time = new_time

    def get_value(self, unit):
        """Return the value of the unit at the current time.
        This model uses black scholes model for one asset, return its value using the simulated array.
        For any other asset that may exist in the timetable, just return the default implementation in
        the model base (i.e. simply return the forwards)."""

        if unit == self.asset:
            return self.spot * np.exp(self.x_vec)
        else:
            return None


## Create the Model class
We will now create the model class. In this case all we have to do is specify the state_class to be used by this model.

In [ ]:
class HestonAESMC(MCModel):

    def state_class(self):
        return HestonAESMCState

## Create a vanilla option contract

In [ ]:
strike = 100
ticker = "EQ"
timetable = Option(
    "USD", ticker, strike=strike, maturity=datetime(2024, 12, 31), is_call=True
).timetable()
print(timetable["events"].to_pandas())

  track                      time op  quantity unit
0       2024-12-31 00:00:00+00:00  >       0.0  USD
1       2024-12-31 00:00:00+00:00  +    -100.0  USD
2       2024-12-31 00:00:00+00:00  +       1.0   EQ


## Create Dataset
Create the dataset, with MC params, discounts and fwds as in previous examples. Add the parameters needed by our model.

In [ ]:
times = np.array([0.0, 5.0])
rates = np.array([0.1, 0.1])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))


spot = 100.0
div_rate = 0.0
fwds = spot * np.exp((rates - div_rate) * times)
fwd_data = ("FORWARDS", np.column_stack((times, fwds)))

dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(datetime(2023, 12, 31)).value,
    "ASSETS": {"USD": discount_data, ticker: fwd_data},
    "MC": {"PATHS": 2_500, "TIMESTEP": 1 / 1000, "SEED": 1, "FLAGS": Stats.CASHFLOW},
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "VOL_OF_VAR": 1.0,
        "MEANREV": 0.5,
        "CORRELATION": -0.9,
    },
}

## Calculate Single Option Price

In [ ]:
model = HestonAESMC()
price, stats = model.price(timetable, dataset)
print(f"price: {price:11.6f}")

price:   12.302057


## Generate Multiple Option prices

- define a contract which is a series of forwards paying at given expiration dates.
- use the cashflow stats to get the cashflow values at all paths
- reconstruct option prices for different strikes and maturities.

In [ ]:
   
expirations = [datetime(2024, 3, 31), datetime(2024, 6, 30), datetime(2024, 12, 31)]
strikes = np.array([0.8, 0.9, 1.0, 1.1, 1.2]) * spot
is_call = True
# Create a timetable that pays forwards at given expirations
events = [
    {
        "track": "",
        "time": dt,
        "op": "+",
        "quantity": 1,
        "unit": ticker,
    }
    for dt in expirations
]

events_table = pa.RecordBatch.from_pylist(events, schema=TS_EVENT_SCHEMA)
fwd_timetable = {"events": events_table, "expressions": {}}

discounter = discounter_from_dataset(dataset)

_, stats = model.price(fwd_timetable, dataset)
# cashflows for track 0, all events
cf = stats["CASHFLOW"][0]

asset_fwds = Forwards(dataset["ASSETS"][ticker])

iv_mat = np.zeros((len(expirations), len(strikes)))
for i, exp in enumerate(expirations):
    prc_ts = dataset["PRICING_TS"]
    # Get Time in years from the millisecond timestamps
    T = (py_to_ts(exp).value - prc_ts) / (365.25 * 24 * 3600 * 1e3)
    df = discounter.discount(T)
    fwd = asset_fwds.forward(T)

    # calculate prices (value as of expiration date)
    event_cf = cf[i] / df
    strikes_c = strikes[..., None]  # Turn into a column vector
    pay = event_cf - strikes_c
    prices = np.maximum(pay, 0).mean(axis=1)

    print(prices)

[2.26848069e+01 1.33881814e+01 4.90740357e+00 1.26416265e-01
 1.37369257e-03]
[25.73005316 16.55131317  7.80295685  0.86197359  0.03006135]
[31.62198349 22.3991109  13.48577577  5.15373019  0.43654451]
